In [53]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [54]:
api_key = input("Please enter your YouTube Data API key: ")
channel_id='UCtAXJ4DNpshfVjLIMq9pIRw'
youtube= build('youtube','v3',developerKey=api_key)

In [55]:
def get_channel(youtube, channel_id):
    all_data = []
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id
    )
    response = request.execute() 
    
    # Check if 'items' key exists in the response
    if 'items' in response:
        for i in range(len(response['items'])):
            data = dict(
                channelName=response['items'][i]['snippet']['title'],
                subscribers=response['items'][i]['statistics']['subscriberCount'],
                views=response['items'][i]['statistics']['viewCount'],
                totalVideos=response['items'][i]['statistics']['videoCount'],
                playlistId=response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
            )
            all_data.append(data)
    else:
        print("No 'items' found in response.")
    
    return pd.DataFrame(all_data)



In [56]:
channel_stats=get_channel(youtube,channel_id)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Rohit & Kanupriya,2280000,2170605651,326,UUtAXJ4DNpshfVjLIMq9pIRw


In [57]:
playlistid=channel_stats['playlistId']
playlistid

0    UUtAXJ4DNpshfVjLIMq9pIRw
Name: playlistId, dtype: object

In [58]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [59]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    
len(Video_ids)

326

In [70]:
def get_comments_in_videos(youtube, video_ids):
    all_comments = []

    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
            )
            response = request.execute()
        
            for comment in response['items']:
                data = {
                    'videoId': video_id,
                    'commenter_name': comment['snippet']['topLevelComment']['snippet']['authorDisplayName'],        
                    'comments': comment['snippet']['topLevelComment']['snippet']['textOriginal'],
                    'commentProfileImageUrl': comment['snippet']['topLevelComment']['snippet']['authorProfileImageUrl']
                }
                all_comments.append(data)
                
                # Add replies
               

        except Exception as e: 
            # When an error occurs - most likely because comments are disabled on a video
            print(f'Could not get comments for video {video_id}: {e}')
        
    return pd.DataFrame(all_comments)


In [71]:
comments = get_comments_in_videos(youtube, video_ids)
comments

,videoId,commenter_name,comments,commentProfileImageUrl
0,yXoqz_GOJxw,@RohitandKanu,"Pehla Nasha, Pehla Scam!! 😂🤣 Do subscribe if y...",https://yt3.ggpht.com/ciKJQ6_fiiQU3caIT8GWU1-5...
1,yXoqz_GOJxw,@MAkInspirational,Behen dikki khuli hi chhor di😂😂😂😂,https://yt3.ggpht.com/ytc/AIdro_l9hYEqOSgKMxnK...
2,yXoqz_GOJxw,@keerthichandra376,I thought she'll tip him in the end and say th...,https://yt3.ggpht.com/ytc/AIdro_mbubIy_N4p_KSr...
3,yXoqz_GOJxw,@oppooppo-ol6ig,But where is hot guy ?? 🤡,https://yt3.ggpht.com/ytc/AIdro_mty4LOIvUzXpPr...
4,yXoqz_GOJxw,@l46565,Why did the acting start even before seeing hi...,https://yt3.ggpht.com/ytc/AIdro_m1mcPKiKrq_FPI...
...,...,...,...,...
6356,xxwNglca0Bo,@dmcindianfan5374,Just went back 10 years ago 😂😅,https://yt3.ggpht.com/Q8dsv0uzwkxGyb8JG9uUoF5L...
6357,xxwNglca0Bo,@coolbreeze1262,Is the record still to her ? ❤,https://yt3.ggpht.com/ytc/AIdro_k2MBvG9XLwRN9I...
6358,xxwNglca0Bo,@Notebook-ur1st,Wow!! She looks different 😳 \nI follow them fo...,https://yt3.ggpht.com/oKSbB0Ayy5knZP0XLiLEStDe...
6359,xxwNglca0Bo,@Prathikraj_mysorean,one can see Rohit's Journey 😂,https://yt3.ggpht.com/D7jURbUnRQ-rWwSYl8YLWMPL...


In [72]:
comments.to_csv('Data\\commentsData.csv', index=False)